based on https://en.wikipedia.org/wiki/Lambda_calculus#Encoding_datatypes

## Helpers

In [1]:
def _increment(x):
    return x + 1

def to_int(church_numeral):
    return church_numeral(_increment)(0)

def check_numeral(church_numeral, numeral):
    assert to_int(church_numeral) == numeral

import inspect

def get_definition(f):
    return inspect.getsource(f).split("=")[1].strip()


    

## Arithmetics

Numerals:

*   0: λf.λx.x
*   1: λf.λx.f x
*   2: λf.λx.f (f x)
*   3: λf.λx.f (f (f x))

In [2]:
_ZERO = lambda f: lambda x: x
_ONE = lambda f: lambda x: f(x)
_TWO = lambda f: lambda x: f(f(x))
_THREE = lambda f: lambda x: f(f(f(x)))

check_numeral(_ZERO, 0)
check_numeral(_ONE, 1)
check_numeral(_TWO, 2)
check_numeral(_THREE, 3)

SUCC := λn.λf.λx.f (n f x)

In [3]:
_SUCC = lambda n: lambda f: lambda x: f(n(f)(x))

check_numeral(_SUCC(_ZERO), 1)
check_numeral(_SUCC(_SUCC(_ZERO)), 2)
check_numeral(_SUCC(_SUCC(_SUCC(_ZERO))), 3)

PLUS := λm.λn.λf.λx.m f (n f x)

In [4]:
_PLUS = lambda m: lambda n: lambda f: lambda x: m(f)(n(f)(x))
_FIVE = _SUCC(_SUCC(lambda f: lambda x: f(f(f(x)))))

check_numeral(_PLUS(_THREE)(_ZERO), 3)
check_numeral(_PLUS(_THREE)(_ONE), 4)
check_numeral(_PLUS(_THREE)(_FIVE), 8)

MULT := λm.λn.λf.m (n f)

In [5]:
_MULT = lambda m: lambda n: lambda f: m(n(f))

check_numeral(_MULT(_THREE)(_ZERO), 0)
check_numeral(_MULT(_THREE)(_ONE), 3)
check_numeral(_MULT(_THREE)(_FIVE), 15)

POW := λb.λe.e b

In [6]:
_POWER = lambda m: lambda n: n(m)

check_numeral(_POWER(_THREE)(_ZERO), 1)
check_numeral(_POWER(_THREE)(_ONE), 3)
check_numeral(_POWER(_THREE)(_FIVE), 243)

PRED := λn.λf.λx.n (λg.λh.h (g f)) (λu.x) (λu.u)

In [7]:
_PRED = lambda n: lambda f: lambda x: n(lambda g: lambda h: h(g(f)))(lambda u: x)(lambda u: u)

check_numeral(_PRED(_ONE), 0)
check_numeral(_PRED(_TWO), 1)
check_numeral(_PRED(_THREE), 2)

SUB := λm.λn.n PRED m

In [8]:
_SUB = lambda m: lambda n: n(lambda n: lambda f: lambda x: n(lambda g: lambda h: h(g(f)))(lambda u: x)(lambda u: u))(m)

check_numeral(_SUB(_THREE)(_ZERO), 3)
check_numeral(_SUB(_THREE)(_TWO), 1)
check_numeral(_SUB(_FIVE)(_TWO), 3)

## Logic and predicates

### Booleans

*   TRUE := λx.λy.x
*   FALSE := λx.λy.y

In [9]:
_TRUE = lambda x: lambda y: x
_FALSE = lambda x: lambda y: y

### Operators

*   AND := λp.λq.p q p
*   OR := λp.λq.p p q
*   NOT := λp.λa.λb.p b a
*   IFTHENELSE := λp.λa.λb.p a b

In [10]:
_AND = lambda p: lambda q: p(q)(p)
_OR = lambda p: lambda q: p(p)(q)
_NOT = lambda p: lambda a: lambda b: p(b)(a)
_IFTHENELSE = lambda p: lambda a: lambda b: p(a)(b)

assert _AND(_TRUE)(_TRUE)(1)(0) == 1
assert _AND(_FALSE)(_TRUE)(1)(0) == 0
assert _OR(_FALSE)(_TRUE)(1)(0) == 1
assert _NOT(_FALSE)(1)(0) == 1
assert _NOT(_TRUE)(1)(0) == 0
assert _IFTHENELSE(_TRUE)(42)(33) == 42
assert _IFTHENELSE(_FALSE)(42)(33) == 33

### Basic predicates

ISZERO := λn.n (λx.FALSE) TRUE

In [11]:
_ISZERO = lambda n: n(lambda x: (lambda x: lambda y: y))(lambda x: lambda y: x)

assert _ISZERO(_ZERO)(1)(0) == 1
assert _ISZERO(_TWO)(1)(0) == 0

LEQ := λm.λn.ISZERO (SUB m n)

EQ := (λe.λm.λn.(AND (e m n))(e n m) )LEQ

In [12]:
_LEQ = lambda m: lambda n: (lambda n: n(lambda x: (lambda x: lambda y: y))(lambda x: lambda y: x))\
    ((lambda m: lambda n: n(lambda n: lambda f: lambda x: n(lambda g: lambda h: h(g(f)))(lambda u: x)(lambda u: u))(m))(m)(n))
_EQ = (lambda leq: lambda m: lambda n: (lambda p: lambda q: p(q)(p))(leq(m)(n))(leq(n)(m)))(lambda m: lambda n: (lambda n: n(lambda x: (lambda x: lambda y: y))(lambda x: lambda y: x))\
    ((lambda m: lambda n: n(lambda n: lambda f: lambda x: n(lambda g: lambda h: h(g(f)))(lambda u: x)(lambda u: u))(m))(m)(n)))
    
assert _LEQ(_ZERO)(_ONE)(1)(0) == 1
assert _LEQ(_ONE)(_THREE)(1)(0) == 1
assert _LEQ(_THREE)(_ONE)(1)(0) == 0
assert _LEQ(_ONE)(_ONE)(1)(0) == 1
assert _EQ(_ONE)(_ONE)(1)(0) == 1
assert _EQ(_ONE)(_TWO)(1)(0) == 0

# Pairs

*   PAIR := λx.λy.λf.f x y
*   FIRST := λp.p TRUE
*   SECOND := λp.p FALSE
*   NIL := λx.TRUE
*   NULL := λp.p (λx.λy.FALSE)

In [13]:
_PAIR = lambda x: lambda y: lambda f: f(x)(y)
_FIRST = lambda p: p(lambda x: lambda y: x)
_SECOND = lambda p: p(lambda x: lambda y: y)
_NIL = lambda x: lambda x: lambda y: x
_NULL = lambda p: p(lambda x: lambda y: (lambda x: lambda y: y))

check_numeral(_FIRST(_PAIR(_ONE)(_TWO)), 1)
check_numeral(_SECOND(_PAIR(_ONE)(_TWO)), 2)

# Recursion

In [30]:
Y = lambda f: (lambda g: g(g))(lambda h: f(lambda x: (h(h))(x)))
# factorial
def factorial_of_5():
    return \
    (((lambda f: (lambda g: g(g))(lambda h: f(lambda x: (h(h))(x))))  # Y
      (lambda factorial: # a name to recur
       (lambda n:
           (((lambda n: n(lambda x: (lambda x: lambda y: y))(lambda x: lambda y: x))(n)) # If it's 0
            (lambda _: (lambda f: lambda x: f(x))) # then 1
            (lambda _: (lambda m: lambda n: lambda f: m(n(f))) # else multiply
             (n) # n by
             (factorial( # recursive call to factorial
                (lambda n: lambda f: lambda x: n(lambda g: lambda h: h(g(f)))(lambda u: x)(lambda u: u))(n) # pred of n
        ))))(0))))
     ((lambda f: lambda x: f(f(f(f(f(x)))))))) # 5
        
#lambda_eval(factorial)
assert to_int(factorial_of_5()) == 120

In [15]:
import meta
import astor

def main():
    (lambda x: x)("sds")
    
ast = meta.decompiler.decompile_func(main)
meta.asttools.print_ast(ast)


FunctionDef(args=arguments(args=[], 
                           defaults=[], 
                           kwarg=None, 
                           vararg=None), 
            body=[Expr(value=Call(args=[Str(s='sds')], 
                                  func=Lambda(args=arguments(args=[Name(ctx=Param(), 
                                                                        id='x')], 
                                                             defaults=[], 
                                                             kwarg=None, 
                                                             vararg=None), 
                                              body=Name(ctx=Load(), 
                                                        id='x')), 
                                  keywords=[], 
                                  kwargs=None, 
                                  starargs=None)), 
                  Return(value=Name(ctx=Load(), 
                                    id='None'))], 
      

In [114]:
import collections
import ast
import copy

class LambdaLevelCounter(ast.NodeTransformer):
    def __init__(self, *args, **kwargs):
        super(LambdaLevelCounter, self).__init__(*args, **kwargs)
        self.scope = []
    
    def current_level(self):
        return len(self.scope)
        
    def visit_Lambda(self, node):
        self.scope.append(node.args.args[0].id)
        self.generic_visit(node)
        self.scope.pop()
        return node

class NamesRemover(LambdaLevelCounter):
    def get_de_bruijn_index(self, name):
        for i, varname in enumerate(reversed(self.scope)):
            if varname == name:
                return i
        else:
            return name

    def visit_Name(self, node):
        node.id = self.get_de_bruijn_index(node.id)
        return node

class NamesGenerator(LambdaLevelCounter):    
    def visit_Name(self, node):
        new_id = node.id
        if isinstance(node.id, int):
            new_id = "v_%s" % (self.current_level() - node.id)
        node.id = new_id
        return node

class IndexModifier(LambdaLevelCounter):   
    def __init__(self, inc):
        super(IndexModifier, self).__init__()
        self.inc = inc
        
    def visit_Name(self, node):
        if isinstance(node.id, int) and node.id >= self.current_level():
            node.id += self.inc
        return node

class Reducer(LambdaLevelCounter):
    def __init__(self, substitution):
        super(Reducer, self).__init__()
        self.substitution = copy.deepcopy(substitution)
    
    def visit_Name(self, node):
        if isinstance(node.id, int):
            if node.id == self.current_level():
                newnode = copy.deepcopy(self.substitution)
                node = IndexModifier(node.id).visit(newnode)
            elif node.id > self.current_level():
                node = IndexModifier(-1).visit(node)
        return node


def reduce_lambda_term(ast_node):
    node_type = type(ast_node)

    # For every func call, I take the argument and I replace
    # the variable
    if isinstance(ast_node, ast.Lambda):
        ast_node.body = reduce_lambda_term(ast_node.body)
    elif isinstance(ast_node, ast.Call):
        #ast_node.args[0] = reduce_lambda_term(ast_node.args[0])
        ast_node.func = reduce_lambda_term(ast_node.func)
        ast_node.args[0] = reduce_lambda_term(ast_node.args[0])
        
        if isinstance(ast_node.func, ast.Lambda):
            ast_node = Reducer(ast_node.args[0]).visit(ast_node.func.body)
            ast_node = reduce_lambda_term(ast_node)
    return ast_node

def lambda_eval(func):
    """
    Evaluate a lambda term, enclosed in a function which does not take any argument,
    like
    def main():
        lambda x: x
    
    Every lambda term can be:
    - A Name node (a variable)
    - Lambda node (lambda abstraction), containing a body (another term) and one argument (a Name)
    - A Call node (an application)
    
    The evaluation is done via substitution.
    """
    tree = meta.decompiler.decompile_func(func)
    nameless = NamesRemover().visit(tree.body[0].value)
  
    reduced = reduce_lambda_term(nameless)
    named = NamesGenerator().visit(reduced)
    return astor.codegen.to_source(named)

In [115]:
def t():
    (lambda x: x)("string")
assert lambda_eval(t) == "'string'"

In [117]:
def succ_three():
    (lambda n: lambda f: lambda x: f(n(f)(x)))\
    (lambda f: lambda x: f(f(f(x))))
    
lambda_eval(succ_three)

'(lambda v_1: (lambda v_2: v_1(v_1(v_1(v_1(v_2))))))'

zero is zero

In [118]:
def zero_eq_zero():
    # _EQ
    (lambda leq: lambda m: lambda n: (lambda p: lambda q: p(q)(p))(leq(m)(n))(leq(n)(m)))(lambda m: lambda n: (lambda n: n(lambda x: (lambda x: lambda y: y))(lambda x: lambda y: x))\
    ((lambda m: lambda n: n(lambda n: lambda f: lambda x: n(lambda g: lambda h: h(g(f)))(lambda u: x)(lambda u: u))(m))(m)(n)))\
    (lambda f: lambda x: x)(lambda f: lambda x: x)("true")("false")
    # ZERO                 # ZERO
assert lambda_eval(zero_eq_zero) == "'true'"

In [119]:
def zero_eq_one():
    # _EQ
    ((lambda leq: lambda m: lambda n: (lambda p: lambda q: p(q)(p))(leq(m)(n))(leq(n)(m)))(lambda m: lambda n: (lambda n: n(lambda x: (lambda x: lambda y: y))(lambda x: lambda y: x))\
    ((lambda m: lambda n: n(lambda n: lambda f: lambda x: n(lambda g: lambda h: h(g(f)))(lambda u: x)(lambda u: u))(m))(m)(n)))\
    (lambda f: lambda x: x)(lambda f: lambda x: f(x))("true")("false"))
    # ZERO                  # ONE
assert lambda_eval(zero_eq_one) == "'false'"

recursion with applicative order Y-combinator (call-by-value)

In [120]:
def main():
    #(lambda n: lambda f: lambda x: f(n(f)(x)))(lambda f: lambda x: f(x))
    (lambda x: lambda y: z(x)(lambda u: u(x))) (lambda x: w(x))

def factorial_of_2():
    return \
    (((lambda f: (lambda g: g(g))(lambda h: f(lambda x: (h(h))(x))))  # Y
      (lambda factorial: # a name to recur
       (lambda n:
           (((lambda n: n(lambda x: (lambda x: lambda y: y))(lambda x: lambda y: x))(n)) # If it's 0
            (lambda _: (lambda f: lambda x: f(x))) # then 1
            (lambda _: (lambda m: lambda n: lambda f: m(n(f))) # else multiply
             (n) # n by
             (factorial( # recursive call to factorial
                (lambda n: lambda f: lambda x: n(lambda g: lambda h: h(g(f)))(lambda u: x)(lambda u: u))(n) # pred of n
        ))))(0))))
     ((lambda f: lambda x: x)))# 5
    
lambda_eval(factorial_of_2)

RuntimeError: maximum recursion depth exceeded while calling a Python object